In [ ]:
"""
notebook by - Kartikeya Bhardwaj (twitter: @kartike_ya)
ported from - https://gist.github.com/nateraw/c989468b74c616ebbc6474aa8cdd9e53
"""

'\nnotebook by - Kartikeya Bhardwaj (twitter: @kartike_ya)\nported from - https://gist.github.com/nateraw/c989468b74c616ebbc6474aa8cdd9e53\n'

## Setup & Imports

You'll need to log into huggingface and accept the terms of the licence for this model - see the [model card](https://huggingface.co/CompVis/stable-diffusion-v1-4) for details.

In [21]:
!pip install -q --upgrade transformers diffusers ftfy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.2 MB/s eta 0:00:00


In [22]:
from huggingface_hub import notebook_login
notebook_login()

In [23]:
import torch
from transformers import CLIPTextModel, CLIPTokenizer
from transformers import logging
from diffusers import AutoencoderKL, UNet2DConditionModel, LMSDiscreteScheduler
from tqdm.auto import tqdm
from torch import autocast
from PIL import Image
from matplotlib import pyplot as plt
import numpy
from torchvision import transforms as tfms

# For video display:
from IPython.display import HTML
from base64 import b64encode

# Supress some unnecessary warnings when loading the CLIPTextModel
logging.set_verbosity_error()

# Set device
torch_device = "cuda" if torch.cuda.is_available() else "cpu"

## Loading the models

This code (and that in the next section) comes from the [Huggingface example notebook](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb).

This will download and set up the relevant models and components we'll be using. Let's just run this for now and move on to the next section to check that it all works before diving deeper.

If you've loaded a pipeline, you can also access these components using `pipe.unet`, `pipe.vae` and so on.

In [24]:
# Load the autoencoder model which will be used to decode the latents into image space.
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

# Load the tokenizer and text encoder to tokenize and encode the text.
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14")
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14")

# The UNet model for generating the latents.
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet")

# The noise scheduler
scheduler = LMSDiscreteScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", num_train_timesteps=1000)

# To the GPU we go!
vae = vae.to(torch_device)
text_encoder = text_encoder.to(torch_device)
unet = unet.to(torch_device);

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


unet/config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

In [25]:
height = 512  # default height of Stable Diffusion
width = 512   # default width of Stable Diffusion
batch_size = 1

## Diffusion func

In [26]:
def diffuse(text_embeddings, init_noise):

  num_inference_steps = 30            # Number of denoising steps
  guidance_scale = 7.5                # Scale for classifier-free guidance
  generator = torch.manual_seed(42)   # Seed generator to create the inital latent noise

  # Prep Scheduler
  scheduler.set_timesteps(num_inference_steps)

  # Prep latents
  latents = init_noise
  latents = latents.to(torch_device)
  latents = latents * scheduler.init_noise_sigma # Scaling (previous versions did latents = latents * self.scheduler.sigmas[0]

  # Loop
  with autocast("cuda"):
      for i, t in tqdm(enumerate(scheduler.timesteps)):
          # expand the latents if we are doing classifier-free guidance to avoid doing two forward passes.
          latent_model_input = torch.cat([latents] * 2)
          sigma = scheduler.sigmas[i]
          # Scale the latents (preconditioning):
          # latent_model_input = latent_model_input / ((sigma**2 + 1) ** 0.5) # Diffusers 0.3 and below
          latent_model_input = scheduler.scale_model_input(latent_model_input, t)

          # predict the noise residual
          with torch.no_grad():
              noise_pred = unet(latent_model_input, t, encoder_hidden_states=text_embeddings).sample

          # perform guidance
          noise_pred_uncond, noise_pred_text = noise_pred.chunk(2)
          noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_text - noise_pred_uncond)

          # compute the previous noisy sample x_t -> x_t-1
          # latents = scheduler.step(noise_pred, i, latents)["prev_sample"] # Diffusers 0.3 and below
          latents = scheduler.step(noise_pred, t, latents).prev_sample

  # scale and decode the image latents with vae
  latents = 1 / 0.18215 * latents
  with torch.no_grad():
      image = vae.decode(latents).sample

  # Display
  image = (image / 2 + 0.5).clamp(0, 1)
  image = image.detach().cpu().permute(0, 2, 3, 1).numpy()
  images = (image * 255).round().astype("uint8")
  pil_images = [Image.fromarray(image) for image in images]
  return pil_images[0]

Spherical Linear Interpolation (SLERP)

In [27]:
def slerp(t, v0, v1, DOT_THRESHOLD=0.9995):
    """ helper function to spherically interpolate two arrays v1 v2 """

    if not isinstance(v0, np.ndarray):
        inputs_are_torch = True
        input_device = v0.device
        v0 = v0.cpu().numpy()
        v1 = v1.cpu().numpy()

    dot = np.sum(v0 * v1 / (np.linalg.norm(v0) * np.linalg.norm(v1)))
    if np.abs(dot) > DOT_THRESHOLD:
        v2 = (1 - t) * v0 + t * v1
    else:
        theta_0 = np.arccos(dot)
        sin_theta_0 = np.sin(theta_0)
        theta_t = theta_0 * t
        sin_theta_t = np.sin(theta_t)
        s0 = np.sin(theta_0 - theta_t) / sin_theta_0
        s1 = sin_theta_t / sin_theta_0
        v2 = s0 * v0 + s1 * v1

    if inputs_are_torch:
        v2 = torch.from_numpy(v2).to(input_device)

    return v2

In [28]:
def embed(prompt):
  text_input = tokenizer(prompt, padding="max_length", max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt")
  with torch.no_grad():
      text_embeddings = text_encoder(text_input.input_ids.to(torch_device))[0]
  max_length = text_input.input_ids.shape[-1]
  uncond_input = tokenizer(
      [""] * batch_size, padding="max_length", max_length=max_length, return_tensors="pt"
  )
  with torch.no_grad():
      uncond_embeddings = text_encoder(uncond_input.input_ids.to(torch_device))[0]
  text_embeddings = torch.cat([uncond_embeddings, text_embeddings])
  return text_embeddings

Loop to interpolate between two text embeddings as well as their noise embeddings

In [29]:
import numpy as np
import os

# list of prompts with seed
prompts = [
    (42, "realistic image of a man alone"),
    (1000, "the man meets a woman"),
    (3000, "the man and the woman get married")
]
audio_prompts = [
    "once upon a time, there was a man. He was alone and sad.",
    "One day, he met a woman and he liked her",
    "They finally got married and lived happily"
]

In [3]:
from transformers import pipeline
pipe = pipeline("text-to-speech", model="suno/bark-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/8.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


generation_config.json:   0%|          | 0.00/4.91k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [44]:
from scipy.io.wavfile import write
# length of audio
audio_len = []
if not(os.path.exists('audios')) or not(os.path.isdir('audios')):
  os.makedirs('audios')
for i in range(len(audio_prompts)):
  output = pipe(audio_prompts[i])
  audio_len.append((len(output["audio"][0])/output["sampling_rate"])*1000)
  outpath = os.path.join("./audios", 'audio%06d.wav' % i)
  normalized_audio = np.int16(output["audio"][0] * 32767)
  write(outpath, output["sampling_rate"], normalized_audio)

In [45]:
audio_len

[11760.0, 2333.3333333333335, 3120.0]

Save Video

In [32]:
import os

if not(os.path.exists('frames')) or not(os.path.isdir('frames')):
  os.makedirs('frames')

In [ ]:
total_photos = 0

for i in range(len(prompts) - 1):
    cur_noise = torch.randn(
    (batch_size, unet.in_channels, height // 8, width // 8),
    generator=torch.manual_seed(prompts[i][0]),
    )
    cur_text_embed = embed(prompts[i][1])
    next_noise = torch.randn(
    (batch_size, unet.in_channels, height // 8, width // 8),
    generator=torch.manual_seed(prompts[i+1][0]),
    )
    next_text_embed = embed(prompts[i+1][1])
    num_steps = int(audio_len[i]//83)
    print(num_steps)
    for j, t in enumerate(np.linspace(0, 1, num_steps)):
      print("dreaming... ", i)

      cond_embedding = slerp(float(t), cur_text_embed, next_text_embed)
      init = slerp(float(t), cur_noise, next_noise)
      image = diffuse(cond_embedding, init)
      outpath = os.path.join("./frames", 'frame%06d.jpg' % total_photos)
      image.save(outpath)
      total_photos += 1
# prompt_1 = "an oil paint sketch of a baby cat"
# seed_1 = 42
# text_embed_1 = embed(prompt_1)
# noise_1  = torch.randn(
#   (batch_size, unet.in_channels, height // 8, width // 8),
#   generator=torch.manual_seed(seed_1),
# )

# prompt_2 = " an oil paint sketch of an old tiger"
# seed_2 = 1000
# text_embed_2 = embed(prompt_2)
# noise_2  = torch.randn(
#   (batch_size, unet.in_channels, height // 8, width // 8),
#   generator=torch.manual_seed(seed_2),
# )

# num_steps = 50

# for i, t in enumerate(np.linspace(0, 1, num_steps)):
#   print("dreaming... ", i)

#   cond_embedding = slerp(float(t), text_embed_1, text_embed_2)
#   init = slerp(float(t), noise_1, noise_2)
#   image = diffuse(cond_embedding, init)
#   outpath = os.path.join("./tiger", 'frame%06d.jpg' % i)
#   image.save(outpath)

In [ ]:
# !ffmpeg -r 10 -f image2 -s 512x512 -i ./frames/frame%06d.jpg -vcodec libx264 -crf 10 -pix_fmt yuv420p evolutionx.mp4

In [74]:
import cv2
import os

# Specify paths
def get_video(start, end, video_num):
  image_folder = 'frames'
  video_name = os.path.join("./videos", 'video%06d.mp4' % video_num)
  frame_rate = 12
  images = sorted(os.listdir(image_folder))
  print(start, end)
  end = min(end, len(images))
  images = images[start:end]

  frame = cv2.imread(os.path.join(image_folder, images[0]))
  height, width, layers = frame.shape

  video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (width, height))

  for image in images:
      video.write(cv2.imread(os.path.join(image_folder, image)))

  cv2.destroyAllWindows()
  video.release()


In [91]:
if not(os.path.exists('videos')) or not(os.path.isdir('videos')):
  os.makedirs('videos')

#generate separate video for each transition
start = 0
for i in range(len(audio_len) - 1):
  frames = int(audio_len[i]//83)
  end = start + frames
  get_video(start, end, i)
  start = end + 1

0 141
142 170


In [93]:
# combine video and audio
from moviepy.editor import VideoFileClip, AudioFileClip

if not(os.path.exists('videos_with_audio')) or not(os.path.isdir('videos_with_audio')):
  os.makedirs('videos_with_audio')

videos = [f for f in sorted(os.listdir('videos')) if f[-4:] == '.mp4']
audios = [f for f in sorted(os.listdir('audios')) if f[-4:] == ".wav"]

# Open video and audio clips

for i in range(len(videos)):
    video_path = os.path.join("./videos", videos[i])
    audio_path = os.path.join("./audios", audios[i])
    video_clip = VideoFileClip(video_path)
    audio_clip = AudioFileClip(audio_path)
    final_clip = video_clip.set_audio(audio_clip)
    video_name = os.path.join("./videos_with_audio", 'video%06d.mp4' % i)
    final_clip.write_videofile(video_name)

t:   1%|▏         | 2/141 [00:37<43:55, 18.96s/it, now=None]

Moviepy - Building video ./videos_with_audio/video000000.mp4.
MoviePy - Writing audio in video000000TEMP_MPY_wvf_snd.mp3



t:   1%|▏         | 2/141 [00:38<44:16, 19.11s/it, now=None]

MoviePy - Done.
Moviepy - Writing video ./videos_with_audio/video000000.mp4




t:   1%|▏         | 2/141 [00:42<48:55, 21.12s/it, now=None]

Moviepy - Done !
Moviepy - video ready ./videos_with_audio/video000000.mp4
Moviepy - Building video ./videos_with_audio/video000001.mp4.
MoviePy - Writing audio in video000001TEMP_MPY_wvf_snd.mp3



t:   1%|▏         | 2/141 [00:42<49:02, 21.17s/it, now=None]

MoviePy - Done.
Moviepy - Writing video ./videos_with_audio/video000001.mp4




t:   1%|▏         | 2/141 [00:43<50:40, 21.87s/it, now=None]

Moviepy - Done !
Moviepy - video ready ./videos_with_audio/video000001.mp4


In [94]:
# concatinate all videos

from moviepy.editor import concatenate_videoclips
videos = [f for f in sorted(os.listdir('videos_with_audio')) if f[-4:] == '.mp4']
clips =  [VideoFileClip(os.path.join("./videos_with_audio", video)) for video in videos]
final_clip = concatenate_videoclips(clips)
final_clip.write_videofile("combined_video.mp4")

t:   1%|▏         | 2/141 [01:22<1:35:32, 41.24s/it, now=None]

Moviepy - Building video combined_video.mp4.
MoviePy - Writing audio in combined_videoTEMP_MPY_wvf_snd.mp3



t:   1%|▏         | 2/141 [01:23<1:36:13, 41.53s/it, now=None]

MoviePy - Done.
Moviepy - Writing video combined_video.mp4




t:  82%|████████▏ | 141/171 [00:04<00:01, 18.36it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file ./videos_with_audio/video000000.mp4, 786432 bytes wanted but 0 bytes read,at frame 141/142, at time 11.75/11.81 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+


t:  99%|█████████▉| 170/171 [00:05<00:00, 19.84it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file ./videos_with_audio/video000001.mp4, 786432 bytes wanted but 0 bytes read,at frame 27/29, at time 2.25/2.38 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+

  warnings.warn("Warning: in file %s, "%(self.filename)+


t:   1%|▏         | 2/141 [01:30<1:45:23, 45.49s/it, now=None]

Moviepy - Done !
Moviepy - video ready combined_video.mp4
